## Step 4 - Administrative summaries

In this notebook, we will import and manipulate the OD matrix - which should have been calculated separately in the previous steps. We assume this has been done, and in turn, that the relevant OD matrix files are under the "result" folder in a sbufolder labeled by the data production date.


In [1]:
# Basics
import os, sys, time
import importlib
# importlib.reload(gn)

# Data tools

import pandas as pd
from pandas.api.types import CategoricalDtype
import geopandas as gpd

import rasterio
from rasterio import features
from shapely.wkt import loads
from shapely import wkt
import numpy as np

import re

# charting tools

import palettable
from functools import reduce
from plotnine import *
from mizani.formatters import percent_format

# printing dicts in easy to read fashion
from pprint import pprint


### Define stuff

Fixed parameters

In [2]:
simplif_meters = 25

In [3]:
source_epsg = 4326
target_epsg = 3106

In [4]:
# scenario = 'Current'
scenario = 'Padma'

In [5]:
# Production date for outputs being used

# prod_date = '210312'
prod_date = '210329'

In [6]:
# WorldPop data determinants

constraint_status = 'constrained'
# constraint_status = 'unconstrained'
wp_res = 100
# wp_res = 250
# wp_res = '1k'

Set path locations

In [7]:
input_pth = r'inputs'
chart_pth = r'charts'
fin_pth = r'final'
res_pth = r'results'
res_prod_pth = f'results\\{prod_date}'
table_pth = r'tables'

adm_pth = r'../../../GEO/Boundaries'
geo_pth = r'../../../GEO'

In [8]:
# make final prod_date subfolder

if not os.path.exists(os.path.join(fin_pth,prod_date)):
    os.makedirs(os.path.join(fin_pth,prod_date))

# make results prod_date folder

if not os.path.exists(os.path.join(res_pth,prod_date)):
    os.makedirs(os.path.join(res_pth,prod_date))
    os.makedirs(os.path.join(res_pth,prod_date,'spatial'))
    os.makedirs(os.path.join(res_pth,prod_date,'tables'))

Load in admin data

In [9]:
# load admin spatial data

adm2 = gpd.read_file(os.path.join(geo_pth,'Boundaries/bd_adm_bbs.gpkg'),layer='bd_adm2_wgs84')
adm2.crs = 'epsg:4326'

adm3 = gpd.read_file(os.path.join(geo_pth,'Boundaries/bd_adm_bbs.gpkg'),layer='bd_adm3_wgs84')
adm3.crs = 'epsg:4326'

adm4 = gpd.read_file(os.path.join(geo_pth,'Boundaries/bd_adm_bbs.gpkg'),layer='bd_adm4_wgs84')
adm4.crs = 'epsg:4326'

adm5 = gpd.read_file(os.path.join(geo_pth,'Boundaries/bd_adm_bbs.gpkg'),layer='bd_adm5_wgs84')
adm5.crs = 'epsg:4326'

In [10]:
# Fix types for later joining
adm2['adm2_pcode'] = adm2['adm2_pcode'].astype(str)
adm3['adm3_pcode'] = adm3['adm3_pcode'].astype(str)
adm4['adm4_pcode'] = adm4['adm4_pcode'].astype(str)
adm5['adm5_pcode'] = adm5['adm5_pcode'].astype(str)


In [11]:
adm3['adm3_en'] = adm3['adm3_en'].str.title()

Dest list for looping

In [12]:
dests = ["All_cities","Dhaka_Chitt","Minor_cities",\
         "Dry_ports","River_ports","Deep_sea_ports",\
         "All_SEZs","Active_SEZs"]

### Prepare OD data

Load in all the OD matrices for all scenarios

In [13]:
Pop_Origins_dct = {}
City_Origins_dct = {}
All_SEZ_Origins_dct = {}
Active_SEZ_Origins_dct = {}

for dest in dests:

    # DFs
    PO_df = f'final_od_grid_{scenario}_{dest}_PopOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
    CO_df = f'final_od_grid_{scenario}_{dest}_CityOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
    AllSEZ_df = f'final_od_grid_{scenario}_{dest}_All_SEZ_Origins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
    ActSEZ_df = f'final_od_grid_{scenario}_{dest}_Active_SEZ_Origins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'

    # Key strings
    PO_str = scenario + '_PopOrigins_' + dest
    CO_str = scenario + '_CityOrigins_' + dest
    AllSEZ_str = scenario + '_All_SEZ_Origins_' + dest
    ActSEZ_str = scenario + '_Act_SEZ_Origins_' + dest

    # Load into dcts
    Pop_Origins_dct[PO_str] = PO_df
    City_Origins_dct[CO_str] = CO_df
    All_SEZ_Origins_dct[AllSEZ_str] = AllSEZ_df
    Active_SEZ_Origins_dct[ActSEZ_str] = ActSEZ_df

In [14]:
from pprint import pprint
pprint(Pop_Origins_dct)

{'Padma_PopOrigins_Active_SEZs': 'final_od_grid_Padma_Active_SEZs_PopOrigins_constrained_100m_res_25m_simplification.csv',
 'Padma_PopOrigins_All_SEZs': 'final_od_grid_Padma_All_SEZs_PopOrigins_constrained_100m_res_25m_simplification.csv',
 'Padma_PopOrigins_All_cities': 'final_od_grid_Padma_All_cities_PopOrigins_constrained_100m_res_25m_simplification.csv',
 'Padma_PopOrigins_Deep_sea_ports': 'final_od_grid_Padma_Deep_sea_ports_PopOrigins_constrained_100m_res_25m_simplification.csv',
 'Padma_PopOrigins_Dhaka_Chitt': 'final_od_grid_Padma_Dhaka_Chitt_PopOrigins_constrained_100m_res_25m_simplification.csv',
 'Padma_PopOrigins_Dry_ports': 'final_od_grid_Padma_Dry_ports_PopOrigins_constrained_100m_res_25m_simplification.csv',
 'Padma_PopOrigins_Minor_cities': 'final_od_grid_Padma_Minor_cities_PopOrigins_constrained_100m_res_25m_simplification.csv',
 'Padma_PopOrigins_River_ports': 'final_od_grid_Padma_River_ports_PopOrigins_constrained_100m_res_25m_simplification.csv'}


In [15]:
# current_PopOrig_all_cities = f'final_od_grid_{scenario}_All_cities_PopOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_PopOrig_minor_cities = f'final_od_grid_{scenario}_Minor_cities_PopOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_PopOrig_DhakaChitt = f'final_od_grid_{scenario}_Dhaka_Chitt_PopOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_PopOrig_dry_ports = f'final_od_grid_{scenario}_Dry_ports_PopOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_PopOrig_river_ports = f'final_od_grid_{scenario}_River_ports_PopOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_PopOrig_deep_sea_ports = f'final_od_grid_{scenario}_Deep_sea_ports_PopOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_PopOrig_all_SEZs = f'final_od_grid_{scenario}_Active_SEZs_PopOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_PopOrig_Active_SEZs = f'final_od_grid_{scenario}_All_SEZs_PopOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'

# current_CityOrig_all_cities = f'final_od_grid_{scenario}_All_cities_CityOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_CityOrig_minor_cities = f'final_od_grid_{scenario}_Minor_cities_CityOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_CityOrig_DhakaChitt = f'final_od_grid_{scenario}_Dhaka_Chitt_CityOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_CityOrig_dry_ports = f'final_od_grid_{scenario}_Dry_ports_CityOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_CityOrig_river_ports = f'final_od_grid_{scenario}_River_ports_CityOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_CityOrig_deep_sea_ports = f'final_od_grid_{scenario}_Deep_sea_ports_CityOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_CityOrig_all_SEZs = f'final_od_grid_{scenario}_Active_SEZs_CityOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
# current_CityOrig_Active_SEZs = f'final_od_grid_{scenario}_All_SEZs_CityOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'

In [16]:
# Pop_Origins_dct = {'current_PopOrig_All_cities' : current_PopOrig_all_cities, \
#                 'current_PopOrig_Minor_cities' : current_PopOrig_minor_cities, \
#                 'current_PopOrig_DhakaChitt' : current_PopOrig_DhakaChitt, \
#                 'current_PopOrig_Dry_ports' : current_PopOrig_dry_ports,  \
#                 'current_PopOrig_River_ports' : current_PopOrig_river_ports,  \
#                 'current_PopOrig_Deep_sea_ports' : current_PopOrig_deep_sea_ports, \
#                 'current_PopOrig_All_SEZs' : current_PopOrig_all_SEZs,  \
#                 'current_PopOrig_Active_SEZs' : current_PopOrig_Active_SEZs }

# City_Origins_dct = {'current_CityOrig_All_cities' : current_CityOrig_all_cities, \
#                 'current_CityOrig_Minor_cities' : current_CityOrig_minor_cities, \
#                 'current_CityOrig_DhakaChitt' : current_CityOrig_DhakaChitt, \
#                 'current_CityOrig_Dry_ports' : current_CityOrig_dry_ports,  \
#                 'current_CityOrig_River_ports' : current_CityOrig_river_ports,  \
#                 'current_CityOrig_Deep_sea_ports' : current_CityOrig_deep_sea_ports, \
#                 'current_CityOrig_All_SEZs' : current_CityOrig_all_SEZs,  \
#                 'current_CityOrig_Active_SEZs' : current_CityOrig_Active_SEZs}

### Basic data loading and manipuation

#### Origins

Load in the WorldPop data from one output layer

In [17]:
pop_orig_pts = pd.read_csv(os.path.join(fin_pth, prod_date,\
                                        f'final_od_grid_{scenario}_All_cities_PopOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'))

pop_orig_pts = pop_orig_pts[['O_ID','Row_ID','VALUE','geometry']]
pop_orig_pts['geometry'] = pop_orig_pts['geometry'].apply(wkt.loads)
pop_orig_pts.set_geometry('geometry')
pop_orig_gdf = gpd.GeoDataFrame(pop_orig_pts,crs = 'epsg:4326')
# pop_orig_gdf = pop_orig_gdf.to_crs(4326)

In [18]:
# city_orig_pts = pd.read_csv(os.path.join(fin_pth, prod_date,\
#                                         f'final_od_grid_{scenario}_All_cities_CityOrigins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'))

# city_orig_pts = city_orig_pts[['O_ID','Row_ID','Pop_all_2011','Pop_all_2021','geometry']]
# city_orig_pts['geometry'] = city_orig_pts['geometry'].apply(wkt.loads)
# city_orig_pts.set_geometry('geometry')
# city_orig_gdf = gpd.GeoDataFrame(city_orig_pts,crs = 'epsg:3106')
# city_orig_gdf = city_orig_gdf.to_crs(4326)

In [19]:
All_SEZ_orig_pts = pd.read_csv(os.path.join(fin_pth, prod_date,\
                                        f'final_od_grid_{scenario}_All_cities_All_SEZ_Origins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'))

All_SEZ_orig_pts = All_SEZ_orig_pts[['O_ID','Row_ID','VALUE','geometry']]
All_SEZ_orig_pts['geometry'] = All_SEZ_orig_pts['geometry'].apply(wkt.loads)
All_SEZ_orig_pts.set_geometry('geometry')
All_SEZ_orig_gdf = gpd.GeoDataFrame(All_SEZ_orig_pts,crs = 'epsg:4326')

In [20]:
Active_SEZ_orig_pts = pd.read_csv(os.path.join(fin_pth, prod_date,\
                                        f'final_od_grid_{scenario}_All_cities_Active_SEZ_Origins_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'))

Active_SEZ_orig_pts = Active_SEZ_orig_pts[['O_ID','Row_ID','VALUE','geometry']]
Active_SEZ_orig_pts['geometry'] = Active_SEZ_orig_pts['geometry'].apply(wkt.loads)
Active_SEZ_orig_pts.set_geometry('geometry')
Active_SEZ_orig_gdf = gpd.GeoDataFrame(Active_SEZ_orig_pts,crs = 'epsg:4326')

Merge all scenario layers onto single origin gdfs

In [21]:
def consolidate_to_origins(orig_fil,dct_key,dct_lyr):

    print(dct_key)

    # read in the CSVs, convert to geometry

    nn_lyr = pd.read_csv(os.path.join(fin_pth,prod_date,dct_lyr))
    
    # create a dict from the Row_ID and the TT to the nearest node, then map that to the origins file
    
    temp_mins_dct = dict(zip(nn_lyr.Row_ID,nn_lyr.PLOT_TIME_MINS))
    orig_fil[f'{dct_key}_mins'] = orig_fil['Row_ID'].map(temp_mins_dct)

    
def NN_adm_aggregates(orig_fil,dct_key):

#     # Start timer
#     func_start = time.time()
    
    print(dct_key)
    
    # calculate other admin pcodes from adm5

    orig_fil['adm1_pcode'] = orig_fil['adm4_pcode'].str[0:2]
    orig_fil['adm2_pcode'] = orig_fil['adm4_pcode'].str[0:4]
    orig_fil['adm3_pcode'] = orig_fil['adm4_pcode'].str[0:6]

    # weight accessibility info by population

    orig_fil['adm2_pop'] = orig_fil[orig_fil[f'{dct_key}_mins'] < 150000].groupby('adm2_pcode')['VALUE'].transform(np.nansum)
    orig_fil['adm3_pop'] = orig_fil[orig_fil[f'{dct_key}_mins'] < 150000].groupby('adm3_pcode')['VALUE'].transform(np.nansum)
    orig_fil['adm4_pop'] = orig_fil[orig_fil[f'{dct_key}_mins'] < 150000].groupby('adm4_pcode')['VALUE'].transform(np.nansum)
    orig_fil['adm5_pop'] = orig_fil[orig_fil[f'{dct_key}_mins'] < 150000].groupby('adm5_pcode')['VALUE'].transform(np.nansum)

    orig_fil[f'{dct_key}_mins_WT_adm2'] = (orig_fil[f'{dct_key}_mins'] * (orig_fil['VALUE'] / orig_fil['adm2_pop']))
    orig_fil[f'{dct_key}_mins_WT_adm3'] = (orig_fil[f'{dct_key}_mins'] * (orig_fil['VALUE'] / orig_fil['adm3_pop']))
    orig_fil[f'{dct_key}_mins_WT_adm4'] = (orig_fil[f'{dct_key}_mins'] * (orig_fil['VALUE'] / orig_fil['adm4_pop']))
    orig_fil[f'{dct_key}_mins_WT_adm5'] = (orig_fil[f'{dct_key}_mins'] * (orig_fil['VALUE'] / orig_fil['adm5_pop']))

#     # Report time

#     func_end = time.time()
#     print('time elapsed for aggregation function')
#     print(str((func_end - func_start) / 60) + ' minutes')

Process and export the data

In [22]:
# iterate through the dict

for key, layer in Pop_Origins_dct.items():
    consolidate_to_origins(orig_fil=pop_orig_gdf,dct_key=key,dct_lyr=layer)
    
# spatial join admin information

pop_orig_gdf = gpd.sjoin(pop_orig_gdf,adm5[['geometry','adm5_pcode']],op="within")
pop_orig_gdf = pop_orig_gdf.drop('index_right',axis=1)
pop_orig_gdf = gpd.sjoin(pop_orig_gdf,adm4[['geometry','adm4_pcode']],op="within") # codes in adm4 and adm5 differ in critical places so must be joined in
pop_orig_gdf = pop_orig_gdf.drop('index_right',axis=1)

# Aggregate by admin level 

for key in Pop_Origins_dct.keys():
    NN_adm_aggregates(orig_fil=pop_orig_gdf,dct_key=key)


Padma_PopOrigins_All_cities
Padma_PopOrigins_Dhaka_Chitt
Padma_PopOrigins_Minor_cities
Padma_PopOrigins_Dry_ports
Padma_PopOrigins_River_ports
Padma_PopOrigins_Deep_sea_ports
Padma_PopOrigins_All_SEZs
Padma_PopOrigins_Active_SEZs
Padma_PopOrigins_All_cities
Padma_PopOrigins_Dhaka_Chitt
Padma_PopOrigins_Minor_cities
Padma_PopOrigins_Dry_ports
Padma_PopOrigins_River_ports
Padma_PopOrigins_Deep_sea_ports
Padma_PopOrigins_All_SEZs
Padma_PopOrigins_Active_SEZs


In [23]:
# iterate through the dict to consolidate travel stats per origin

# for key, layer in City_Origins_dct.items():
#     consolidate_to_origins(orig_fil=city_orig_gdf,dct_key=key,dct_lyr=layer)

for key, layer in All_SEZ_Origins_dct.items():
    consolidate_to_origins(orig_fil=All_SEZ_orig_gdf,dct_key=key,dct_lyr=layer)

for key, layer in Active_SEZ_Origins_dct.items():
    consolidate_to_origins(orig_fil=Active_SEZ_orig_gdf,dct_key=key,dct_lyr=layer)


Padma_All_SEZ_Origins_All_cities
Padma_All_SEZ_Origins_Dhaka_Chitt
Padma_All_SEZ_Origins_Minor_cities
Padma_All_SEZ_Origins_Dry_ports
Padma_All_SEZ_Origins_River_ports
Padma_All_SEZ_Origins_Deep_sea_ports
Padma_All_SEZ_Origins_All_SEZs
Padma_All_SEZ_Origins_Active_SEZs
Padma_Act_SEZ_Origins_All_cities
Padma_Act_SEZ_Origins_Dhaka_Chitt
Padma_Act_SEZ_Origins_Minor_cities
Padma_Act_SEZ_Origins_Dry_ports
Padma_Act_SEZ_Origins_River_ports
Padma_Act_SEZ_Origins_Deep_sea_ports
Padma_Act_SEZ_Origins_All_SEZs
Padma_Act_SEZ_Origins_Active_SEZs


In [24]:
# city_orig_gdf.to_file(os.path.join(res_prod_pth,'spatial',f'{scenario}_origins_{wp_res}m_TTs.gpkg'),layer=f"CityOrigins",driver='GPKG',OVERWRITE=YES)
All_SEZ_orig_gdf.to_file(os.path.join(res_prod_pth,'spatial',f'{scenario}_origins_{wp_res}m_TTs.gpkg'),layer=f"All_SEZ_Origins",driver='GPKG')
Active_SEZ_orig_gdf.to_file(os.path.join(res_prod_pth,'spatial',f'{scenario}_origins_{wp_res}m_TTs.gpkg'),layer=f"Active_SEZ_Origins",driver='GPKG')

In [25]:
pop_orig_gdf.to_file(os.path.join(res_prod_pth,'spatial',f'{scenario}_origins_{wp_res}m_TTs.gpkg'),layer=f"PopOrigins",driver='GPKG')

### Administrative aggregates


Population weighting origin grid data by admin and then summarizing per admin.

In [26]:
# aggregate and weight indicators by admin unit using WorldPop Origins

for key in Pop_Origins_dct.keys():

#     # Start timer
    
#     func_start = time.time()

    # define variable for avg_time naming
    
    at = (key + '_avg_time')
    
    # adm2
    
    zila = pop_orig_gdf[pop_orig_gdf[f'{key}_mins'] < 150000].groupby(['adm2_pcode']).agg(
        at = (f'{key}_mins_WT_adm2',np.nansum),    ) 
    
    zila = zila.rename(columns={'at' : at}).reset_index()
    zila['adm2_pcode'] = zila['adm2_pcode'].astype(str)
    zila = zila[['adm2_pcode',at]]
    
    adm2 = pd.merge(adm2,zila,how='left',on=['adm2_pcode'])
    
    # adm3
    
    upz = pop_orig_gdf[pop_orig_gdf[f'{key}_mins'] < 150000].groupby(['adm3_pcode']).agg(
        at = (f'{key}_mins_WT_adm3',np.nansum),    ) 
    
    upz = upz.rename(columns={'at' : at}).reset_index()
    upz['adm3_pcode'] = upz['adm3_pcode'].astype(str)
    upz = upz[['adm3_pcode',at]]
    
    adm3 = pd.merge(adm3,upz,how='left',on=['adm3_pcode'])

    # adm4
    
    union = pop_orig_gdf[pop_orig_gdf[f'{key}_mins'] < 150000].groupby(['adm4_pcode']).agg(
        at = (f'{key}_mins_WT_adm4',np.nansum),    ) 
    
    union = union.rename(columns={'at' : at}).reset_index()
    union['adm4_pcode'] = union['adm4_pcode'].astype(str)
    union = union[['adm4_pcode',at]]
    
    adm4 = pd.merge(adm4,union,how='left',on=['adm4_pcode'])  

    # adm5
    
    mauza = pop_orig_gdf[pop_orig_gdf[f'{key}_mins'] < 150000].groupby(['adm5_pcode']).agg(
        at = (f'{key}_mins_WT_adm5',np.nansum),    ) 
    
    mauza = mauza.rename(columns={'at' : at}).reset_index()
    mauza['adm5_pcode'] = mauza['adm5_pcode'].astype(str)
    mauza = mauza[['adm5_pcode',at]]
    
    adm5 = pd.merge(adm5,mauza,how='left',on=['adm5_pcode'])

#     # Report function time
    
#     func_end = time.time()
#     print(f'time elapsed for summing {key}')
#     print(str((func_end - func_start) / 60) + ' minutes')

Export resulting files

In [27]:
adm2.loc[:, adm2.columns != 'geometry'].to_csv(os.path.join(f'results/{prod_date}/tables/adm2_{scenario}_{constraint_status}_{wp_res}m_res.csv'))
adm3.loc[:, adm3.columns != 'geometry'].to_csv(os.path.join(f'results/{prod_date}/tables/adm3_{scenario}_{constraint_status}_{wp_res}m_res.csv'))
adm4.loc[:, adm4.columns != 'geometry'].to_csv(os.path.join(f'results/{prod_date}/tables/adm4_{scenario}_{constraint_status}_{wp_res}m_res.csv'))
adm5.loc[:, adm5.columns != 'geometry'].to_csv(os.path.join(f'results/{prod_date}/tables/adm5_{scenario}_{constraint_status}_{wp_res}m_res.csv'))

In [28]:
# adm2.to_file(os.path.join(res_prod_pth,f'spatial/adm2_{constraint_status}_{wp_res}m_res.shp'))
# adm3.to_file(os.path.join(res_prod_pth,f'spatial/adm3_{constraint_status}_{wp_res}m_res.shp'))
# adm4.to_file(os.path.join(res_prod_pth,f'spatial/adm4_{constraint_status}_{wp_res}m_res.shp'))
# adm5.to_file(os.path.join(res_prod_pth,f'spatial/adm5_{constraint_status}_{wp_res}m_res.shp'))

In [29]:
adm2.to_file(os.path.join(res_prod_pth,f'spatial/adm_{scenario}_{constraint_status}_{wp_res}m_res.gpkg'),layer='adm2',driver='GPKG')
adm3.to_file(os.path.join(res_prod_pth,f'spatial/adm_{scenario}_{constraint_status}_{wp_res}m_res.gpkg'),layer='adm3',driver='GPKG')
adm4.to_file(os.path.join(res_prod_pth,f'spatial/adm_{scenario}_{constraint_status}_{wp_res}m_res.gpkg'),layer='adm4',driver='GPKG')
adm5.to_file(os.path.join(res_prod_pth,f'spatial/adm_{scenario}_{constraint_status}_{wp_res}m_res.gpkg'),layer='adm5',driver='GPKG')